# Analysis of the benchmarking results

## Setup

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path

import plotly.express as px
import plotly.io as pio


pio.templates["latex"] = pio.templates["plotly_white"].update(
    layout=dict(
        colorway=px.colors.qualitative.D3,
        font=dict(
            family="CMU Typewriter Text, Courier New",
            size=16
        )
    )
)

pio.templates.default = "latex"

In [2]:
RESULTS_PATH = Path(".").parent / "results"
PLOTS_PATH = RESULTS_PATH / "plots"

GRAYSCALE_RESULT_FILE = RESULTS_PATH / "2025_12_19__09_09_24_grayscale_full.csv"
RGB_RESULT_FILE = RESULTS_PATH / "2025_12_20__20_19_37_rgb_full.csv"

In [3]:
grayscale_df = pd.read_csv(GRAYSCALE_RESULT_FILE)
rgb_df = pd.read_csv(RGB_RESULT_FILE)

grayscale_df["color_mode"] = "grayscale"
rgb_df["color_mode"] = "rgb"

In [4]:
df = pd.concat([grayscale_df, rgb_df], ignore_index=True)
df["image_size"] = df["image_name"].str.extract(r"_(\d+)x\d+").astype(int)
df["method_name"] = df["method_name"].str.upper()
df

,method_name,image_name,pcc_clean,psnr_clean,noise_model,circuit_depth,circuit_size,cnot_count,n_qubits,n_shots,...,pcc_noised_0.2,psnr_noised_0.2,pcc_noised_0.5,psnr_noised_0.5,pcc_noised_0.9,psnr_noised_0.9,pcc_noised_1.0,psnr_noised_1.0,color_mode,image_size
0,FRQI,image_000_2x2_,0.999035,32.507875,depolarizing_1q,3,5,20,3,1024,...,0.998460,27.179110,0.986033,29.251819,0.911729,24.960623,0.914864,28.576332,grayscale,2
1,FRQI,image_001_2x2_,0.999762,40.001670,depolarizing_1q,3,5,20,3,1024,...,0.994712,34.283686,0.971934,27.697181,0.725987,29.019228,0.280559,30.001670,grayscale,2
2,FRQI,image_002_2x2_,0.999295,36.298105,depolarizing_1q,3,5,20,3,1024,...,0.996836,26.999730,0.994870,30.786806,0.962834,28.002431,0.859582,30.949941,grayscale,2
3,FRQI,image_003_2x2_,0.999660,36.991370,depolarizing_1q,3,5,20,3,1024,...,0.997011,27.016498,0.993598,26.941546,0.968989,27.619278,0.873709,28.710723,grayscale,2
4,FRQI,image_004_2x2_,0.999498,36.089604,depolarizing_1q,3,5,20,3,1024,...,0.999555,28.966264,0.998318,27.908583,0.955542,26.949810,-0.465259,26.442883,grayscale,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,NCQI,image_003_2x2_,1.000000,inf,depolarizing_1q,3,14,-1,12,1024,...,0.364891,30.820291,0.476435,33.217187,-0.066323,30.125924,0.886058,31.090729,rgb,2
156,NCQI,image_004_2x2_,1.000000,inf,depolarizing_1q,3,14,-1,12,1024,...,0.345491,34.451036,0.581470,42.587932,0.473332,39.577632,0.398323,34.451036,rgb,2
157,NCQI,image_005_2x2_,1.000000,inf,depolarizing_1q,3,14,-1,12,1024,...,0.908086,36.000055,0.874020,31.177446,0.895146,30.733762,0.616408,28.156212,rgb,2
158,NCQI,image_006_2x2_,1.000000,inf,depolarizing_1q,3,14,-1,12,1024,...,1.000000,inf,0.718107,52.902016,0.816006,37.850516,0.819891,40.860816,rgb,2


In [5]:
GROUPBY_COLUMNS = ["color_mode", "method_name", "image_size", "n_qubits"]
NOISE_LEVELS = [0.0, 0.01, 0.1, 0.2, 0.5, 0.9, 1.0]

PCC_COLUMNS = [column for column in df.columns if "pcc" in column]
PSNR_COLUMNS = [column for column in df.columns if "psnr" in column]
METRICS_COLUMNS = PCC_COLUMNS + PSNR_COLUMNS

In [6]:
df_mean = df.groupby(GROUPBY_COLUMNS)[METRICS_COLUMNS].mean().reset_index()
df_mean

,color_mode,method_name,image_size,n_qubits,pcc_clean,pcc_noised_0.01,pcc_noised_0.1,pcc_noised_0.2,pcc_noised_0.5,pcc_noised_0.9,pcc_noised_1.0,psnr_clean,psnr_noised_0.01,psnr_noised_0.1,psnr_noised_0.2,psnr_noised_0.5,psnr_noised_0.9,psnr_noised_1.0
0,grayscale,FRQI,2,3,0.999499,0.998292,0.997778,0.996711,0.989040,0.854069,0.411044,37.133326,30.392412,28.140214,28.343394,28.566030,27.631570,29.675726
1,grayscale,FRQI,4,5,0.993231,0.992358,0.985796,0.978208,0.949487,0.582267,0.008871,30.797725,29.837306,28.659533,28.779550,28.206933,28.264180,27.687429
2,grayscale,FRQI,8,7,0.964293,0.965186,0.936005,0.919216,0.817035,0.255492,0.026827,28.644846,28.573010,28.445872,28.160064,27.643998,28.101947,28.076418
3,grayscale,IFRQI,2,6,1.000000,1.000000,1.000000,1.000000,0.999948,0.999935,0.999962,inf,inf,inf,inf,inf,43.024280,45.688133
4,grayscale,IFRQI,4,8,0.955688,0.949213,0.984126,0.980275,0.966432,0.932721,0.905386,inf,inf,inf,inf,34.464536,30.532228,23.012298
5,grayscale,IFRQI,8,10,0.751898,0.761193,0.728169,0.715488,0.701768,0.679836,0.707593,12.899000,12.966355,12.456656,12.420569,11.950048,11.717931,12.150224
6,grayscale,NEQR,2,10,1.000000,0.999998,0.999997,0.999993,0.999991,0.999989,0.999994,inf,inf,inf,inf,inf,52.646254,inf
7,grayscale,NEQR,4,12,1.000000,0.999997,0.999959,0.999961,0.999966,0.999971,0.999974,inf,inf,50.256817,50.262273,50.863180,51.705956,52.524503
8,grayscale,QUALPI,2,10,1.000000,0.999997,0.999996,0.999993,0.999996,0.999990,0.999994,inf,inf,inf,inf,inf,inf,52.426139
9,grayscale,QUALPI,4,12,1.000000,0.999996,0.999956,0.999958,0.999969,0.999968,0.999969,inf,inf,50.064196,50.346012,51.325163,51.162528,51.569526


### PCC Plots

In [7]:
def _pcc_to_noise(col: str) -> float:
    if col == "pcc_clean":
        return 0.0
    return float(col.replace("pcc_noised_", ""))

pcc_df = df_mean.melt(
    id_vars=GROUPBY_COLUMNS,
    value_vars=PCC_COLUMNS,
    var_name="pcc_metric",
    value_name="pcc",
)
pcc_df["noise_level"] = pcc_df["pcc_metric"].apply(_pcc_to_noise)
pcc_df = pcc_df.sort_values(["method_name", "image_size", "noise_level"])

symbol_map = {2: "circle", 4: "triangle-up", 8: "square"}

In [8]:
pcc_gray = pcc_df[pcc_df["color_mode"] == "grayscale"]

fig_gray = px.line(
    pcc_gray,
    x="noise_level",
    y="pcc",
    title="PCC vs Noise Level — Grayscale",
    color="method_name",
    markers=True,
    symbol="image_size",
    symbol_map=symbol_map,
)
fig_gray.update_traces(line=dict(width=1), marker=dict(size=8))
fig_gray.update_layout(
    xaxis=dict(title="Noise level [-]", tickvals=NOISE_LEVELS),
    yaxis=dict(title="PCC [-]", range=[0, 1.01]),
    width=1000,
    height=600,
    legend_title_text="Method, Image size",
)
fig_gray.show()
fig_gray.write_image(PLOTS_PATH / "pcc_vs_noise_grayscale.pdf")


In [9]:
pcc_rgb = pcc_df[pcc_df["color_mode"] == "rgb"]

fig_rgb = px.line(
    pcc_rgb,
    x="noise_level",
    y="pcc",
    title="PCC vs Noise level — RGB",
    color="method_name",
    markers=True,
    symbol="image_size",
    symbol_map=symbol_map,
    color_discrete_sequence=px.colors.qualitative.D3,
)
fig_rgb.update_traces(line=dict(width=1), marker=dict(size=8))
fig_rgb.update_layout(
    template="plotly_white",
    xaxis=dict(title="Noise level [-]", tickvals=NOISE_LEVELS),
    yaxis=dict(title="PCC [-]", range=[0, 1.01]),
    width=1000,
    height=600,
    legend_title_text="Method, Image size",
)

fig_rgb.show()
fig_rgb.write_image(PLOTS_PATH / "pcc_vs_noise_rgb.pdf")


### PSNR Plots

In [10]:
def _psnr_to_noise(col: str) -> float:
    if col == "psnr_clean":
        return 0.0
    return float(col.replace("psnr_noised_", ""))

psnr_df = df_mean.melt(
    id_vars=GROUPBY_COLUMNS,
    value_vars=PSNR_COLUMNS,
    var_name="psnr_metric",
    value_name="psnr",
)
psnr_df["noise_level"] = psnr_df["psnr_metric"].apply(_psnr_to_noise)

# handle infinite PSNR values by converting them to NaN so plots skip them
psnr_df["psnr"] = psnr_df["psnr"].replace([np.inf, -np.inf], 100)

In [11]:
psnr_gray = psnr_df[psnr_df["color_mode"] == "grayscale"]

fig_gray = px.line(
    psnr_gray,
    x="noise_level",
    y="psnr",
    title="PSNR vs Noise Level — Grayscale",
    color="method_name",
    markers=True,
    symbol="image_size",
    symbol_map=symbol_map,
    color_discrete_sequence=px.colors.qualitative.D3,
)
fig_gray.update_traces(line=dict(width=1), marker=dict(size=8))
fig_gray.update_layout(
    template="plotly_white",
    xaxis=dict(title="Noise level [-]", tickvals=NOISE_LEVELS),
    yaxis=dict(title="PSNR [dB]", range=[10, 60]),
    width=1000,
    height=600,
    legend_title_text="Method, Image size",
)

fig_gray.show()
fig_gray.write_image(PLOTS_PATH / "psnr_vs_noise_grayscale.pdf")


In [12]:
psnr_rgb = psnr_df[psnr_df["color_mode"] == "rgb"]

fig_rgb = px.line(
    psnr_rgb,
    x="noise_level",
    y="psnr",
    title="PSNR vs Noise Level — RGB",
    color="method_name",
    markers=True,
    symbol="image_size",
    symbol_map=symbol_map,
    color_discrete_sequence=px.colors.qualitative.D3,
)
fig_rgb.update_traces(line=dict(width=1), marker=dict(size=8))
fig_rgb.update_layout(
    template="plotly_white",
    xaxis=dict(title="Noise level [-]", tickvals=NOISE_LEVELS),
    yaxis=dict(title="PSNR [dB]", range=[10, 60]),
    width=1000,
    height=600,
    legend_title_text="Method, Image size",
)

fig_rgb.show()
fig_rgb.write_image(PLOTS_PATH / "psnr_vs_noise_rgb.pdf")
